# <font color='Teal'> Inversion Detection</font> <font color='green'>using</font> <font color='red'>Computer Vision</font>
---

### Initialization

In [ ]:
import cv2
import numpy as np
from scipy.optimize import curve_fit
import cv2
import matplotlib.pyplot as plt
import random
import json
import numpy as np
import cv2
import sys
from scipy.spatial.distance import cdist, cosine
from shape_context import ShapeContext
import matplotlib.pyplot as plt
import imutils

### Setting an Helper Function for Finding Dilation Constant

In [ ]:
def dilation_constant_selector(image):
    ContourCount =[]
    kernel_value =0
    for i in range (1,100):
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (i,i))
        dilate = cv2.dilate(image,kernel,iterations=1)
        contours = cv2.findContours(dilate,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        ContourCount.append(len(contours[1]))
#         plt.plot(ContourCount)
#         plt.show()
        if i >=4:
            pixelC, trigg = findCurveFit(ContourCount)
            if trigg == True:
                kernel_value=i
                break
    return kernel_value            

### Defining functions for Curve Fitting

In [ ]:
def sigmoid(x, L ,x0, k, b):

    y = (L / (1 + np.exp(k*(x+x0)))+b)
    return (y)


def isCurveSigmoid(ContourCounts, count):


        xIndex = len(ContourCounts)

        p0 = [max(ContourCounts), np.median(xIndex),1,min(ContourCounts)] # this is an mandatory initial guess

        popt, pcov = curve_fit(sigmoid, list(range(xIndex)), ContourCounts, p0, method='lm', maxfev=10000)

        yVals = sigmoid(list(range(xIndex)), *popt)

    

        # May have to check for a value much less than Median to avoid false positives.
        if np.median(yVals[:3]) - np.median(yVals[-3:]) > 15:
            print('Triggered Event')
            return True
        else:

            return False




def findCurveFit(ContourCount):

    triggerEvent = False

    if isCurveSigmoid(ContourCount, len(ContourCount)):
        print('Event Triggered...')
        triggerEvent = True


    return ContourCount, triggerEvent

### Countour Bounding Box Method

In [ ]:
z=0
s=0

def line_removal(image_path):
    image = image_path
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (60,1))
    verti_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,40))

    hori_line = 255-cv2.morphologyEx(image,cv2.MORPH_CLOSE, hori_kernel,iterations=1)

    result = cv2.add(image,hori_line)

    verti_line = 255 - cv2.morphologyEx(image,cv2.MORPH_CLOSE, verti_kernel,iterations=1)

    result = cv2.add(result,verti_line)
    return result


def word_picker(image):
    arr = []
    c = 0
    out = image.copy()
    line_removed = line_removal(image)
#     plt.imshow(line_removed)
#     plt.show()

    imagem = cv2.bitwise_not(line_removed)
#     plt.imshow(imagem)
#     plt.show()
    k = dilation_constant_selector(imagem)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k,k))
    dilate = cv2.dilate(imagem,kernel,iterations=1)
#     plt.imshow(dilate)
#     plt.show()

    contours = cv2.findContours(dilate, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for i in range(0, len(contours[1])):
        area = cv2.contourArea(contours[1][i])
        x,y,w,h = cv2.boundingRect(contours[1][i])
        #arr.append([x,y,x+w,y+h])
        arr.append([x,y,w,h])
    cv2.imwrite("r.jpg",out)
    arr.sort(key = lambda x: x[2])
#     c = random.choice(arr)
    print(arr)
    print(arr[-1])
    c = arr[-1]
    crop_img = out[c[1]:c[1]+c[3], c[0]:c[0]+c[2]]
    return crop_img

### Functions for Language Identification and Shape Context

In [ ]:
sc = ShapeContext()

def get_contour_bounding_rectangles(gray):
    """
      Getting all 2nd level bouding boxes based on contour detection algorithm.
    """

    print(gray.shape)
    cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    res = []
    for cnt in cnts[1]:
        (x, y, w, h) = cv2.boundingRect(cnt)
        res.append((x, y, x + w, y + h))

    return res

def parse_nums(sc, path):
    global s
    img = np.asarray(path)
    # invert image colors
    img = cv2.bitwise_not(img)
    _, img = cv2.threshold(img, 254, 255, cv2.THRESH_BINARY)
    # making numbers fat for better contour detectiion
    kernel = np.ones((3, 3), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
#     print('After thresholding and dilation...')
#     plt.imshow(img)
#     plt.show()
    # getting our numbers one by one
    rois = get_contour_bounding_rectangles(img)
    grayd = cv2.cvtColor(img.copy(), cv2.COLOR_GRAY2BGR)
    nums = []
    for r in rois:
        grayd = cv2.rectangle(grayd, (r[0], r[1]), (r[2], r[3]),(0, 255, 0), 1)
        nums.append((r[0], r[1], r[2], r[3]))

# #     print('After greying and bounding...')
    # we are getting contours in different order so we need to sort them by x1
    nums = sorted(nums, key=lambda x: x[0])
#     print('bounding box x coords')
    if (z % 3 == 0):
        s = len(nums)
    else:
        pass
    descs = []
    for i, r in enumerate(nums):

        points = sc.get_points_from_img(img[r[1]:r[3], r[0]:r[2]], 15)
        descriptor = sc.compute(points).flatten()
        descs.append(descriptor)
    #print(descs)
    return np.array(descs), s

def match(base, current,s):
    """
      Here we are using cosine diff instead of "by paper" diff, cause it's faster
    """
    res = cdist(base, current.reshape((1, current.shape[0])), metric="cosine")
    # print("min = " + np.argmin(res.reshape(11)))
    char = str(np.argmin(res.reshape(s)))
    # print(char)
#     print(np.min(res.reshape(53)))
    return char, np.min(res.reshape(s))


def language_identification(language_base,testImg):
    d={}
    global z,s
    matchs=[]
    langs=[]
    for language,base_url in language_base.items():
        print(language,base_url)
        baseImage = cv2.imread(base_url,0)
        if len(baseImage.shape) !=2:
            baseImage=cv2.cvtColor(baseImage, cv2.COLOR_BGR2GRAY)
        parsed_base,s = parse_nums(sc, baseImage)
        z+=1
        recognize,s = parse_nums(sc, testImg)
        z+=1
        secondImg = cv2.rotate(testImg,cv2.ROTATE_180)
        recognize_inverted,s = parse_nums(sc, secondImg)
        z+=1
        txt = ""
        matchFactor = 0
        val = 0
        c=0
        for r in recognize:
            c, val = match(parsed_base, r,s)
            txt += c
            matchFactor += val
        txtInverted = ""
        matchFactorInv = 0
        val = 0
        c=0
        for r in recognize_inverted:
            c, val = match(parsed_base, r,s)
            txtInverted += c
            matchFactorInv += val 
        print(matchFactor,matchFactorInv)
#         a = abs(matchFactor-matchFactorInv)
#         d[language]=a
        matchs.append([matchFactor,matchFactorInv])
        langs.append(language)
        langs = list(set(langs))
    
    matc = min(matchs)
    index = matchs.index(matc)
    language_identified = langs[index]

    return language_identified,matc
    


def findUpright(language_base,testImg):
    
    language,match1 = language_identification(language_base,testImg)


    print("\nUpright Text Match Value = " + str(match1[0]))
    print("Flip Text Match Value = " + str(match1[1]))
    
    if (match1[0]> match1[1]):
        result = "inverted"
        lang_1 = language
        return lang_1, result
    else:
        result = "upright"
        lang_1 = language
        return lang_1,result

### Loading the Language Base and Availiable Data

In [ ]:
with open('generated.json') as json_file:
    language_base = json.load(json_file)
    
image = cv2.imread(r'./data/8.png',0)
cropped = word_picker(image)
plt.imshow(cropped)
plt.show()
language, result = findUpright(language_base,cropped)
print(language,result)
# match, result = findUpright(cropped)